In [ ]:
"""

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': [
            # Temperature
            '2m_dewpoint_temperature', '2m_temperature', 'skin_temperature',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4',
            #Lakes
            'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature',
            'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor',
            'lake_total_layer_temperature',
            #Snow
            'snow_albedo', 'snow_cover', 'snow_density',
            'snow_depth', 'snow_depth_water_equivalent', 'snowfall',
            'snowmelt', 'temperature_of_snow_layer',
            #Soil Water
            'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2',
            'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4',
            #Radiation and Heat
            'forecast_albedo', 'surface_latent_heat_flux', 'surface_net_solar_radiation',
            'surface_net_thermal_radiation', 'surface_sensible_heat_flux', 'surface_solar_radiation_downwards',
            'surface_thermal_radiation_downwards',
            #Evaporation and Runoff
            'evaporation_from_bare_soil', 'evaporation_from_open_water_surfaces_excluding_oceans', 'evaporation_from_the_top_of_canopy',
            'evaporation_from_vegetation_transpiration', 'potential_evaporation', 'runoff',
            'snow_evaporation', 'sub_surface_runoff', 'surface_runoff',
            'total_evaporation',
            #Wind, Pressure and Precipitation
            '10m_u_component_of_wind', '10m_v_component_of_wind', 'surface_pressure',
            'total_precipitation',
            #Vegetation
            'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
        ],
        'year': '2023',
        'month': '01',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '06:00', '12:00',
            '18:00',
        ],
        'area': [
            5.39, -6.5, 5.32,
            -6.42,
        ],
        'format': 'grib',
        
    },
    'download.grib')
"""

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

/var/folders/0k/qxvpc_xn6j3gvzhy4sbc0g0r0000gq/T/ipykernel_48660/3143207956.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [9]:

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [5.39077, 5.392571, 5.324686, 5.32367],
	"longitude": [-6.505318, -6.427247, -6.502779, -6.427451],
	"start_date": "1940-01-01",
	"end_date": "2023-12-31",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
	"wind_speed_unit": "ms",
	"timezone": "GMT"
}
responses = openmeteo.weather_api(url, params=params)

In [10]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 5.3778557777404785°E -6.481781005859375°N
Elevation 109.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [11]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesAsNumpy()
daily_sunset = daily.Variables(8).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(19).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)


In [19]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30680 entries, 1 to 30680
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         30680 non-null  datetime64[ns]
 1   weather_code                 30680 non-null  float32       
 2   temperature_2m_max           30680 non-null  float32       
 3   temperature_2m_min           30680 non-null  float32       
 4   temperature_2m_mean          30680 non-null  float32       
 5   apparent_temperature_max     30680 non-null  float32       
 6   apparent_temperature_min     30680 non-null  float32       
 7   apparent_temperature_mean    30680 non-null  float32       
 8   sunrise                      30680 non-null  int64         
 9   sunset                       30680 non-null  int64         
 10  daylight_duration            30680 non-null  float32       
 11  sunshine_duration            30680 non-null  f

In [18]:
daily_dataframe.dropna(inplace=True)
#Remove NA (should only be 1 line for 01/01/1940)

In [21]:
daily_dataframe.iloc[16554]

date                           1985-04-29 00:00:00
weather_code                                  63.0
temperature_2m_max                       30.967501
temperature_2m_min                         22.8675
temperature_2m_mean                      25.877916
apparent_temperature_max                 37.503307
apparent_temperature_min                  27.52281
apparent_temperature_mean                 31.06517
sunrise                                          0
sunset                                           0
daylight_duration                     44286.085938
sunshine_duration                     36364.222656
precipitation_sum                              4.1
rain_sum                                       4.1
snowfall_sum                                   0.0
precipitation_hours                            2.0
wind_speed_10m_max                        2.828427
wind_gusts_10m_max                             6.1
wind_direction_10m_dominant             227.459045
shortwave_radiation_sum        

In [29]:
daily_dataframe.set_index(['date'], inplace=True)

In [30]:
daily_dataframe.groupby(daily_dataframe.index.dayofyear).mean()

,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
date,,,,,,,,,,,,,,,,,,,,
1,38.951809,29.982813,21.949680,24.992262,34.573524,25.946907,29.219023,0.0,0.0,42537.402344,32533.523438,1.540964,1.540964,0.0,2.927711,2.582955,6.420482,210.789490,15.995180,3.334029
2,38.047619,29.934649,21.872149,25.008856,34.530060,25.858582,29.213818,0.0,0.0,42541.695312,31881.541016,1.310714,1.310714,0.0,3.071429,2.559789,6.379762,209.204376,15.987380,3.337215
3,37.761906,30.127506,21.844173,25.079613,34.772186,25.807106,29.281723,0.0,0.0,42546.355469,32545.146484,1.078571,1.078571,0.0,2.666667,2.563935,6.400000,200.890350,16.460714,3.436635
4,30.547619,30.207266,21.758459,25.110939,34.690365,25.655012,29.240042,0.0,0.0,42551.390625,32246.431641,0.777381,0.777381,0.0,2.095238,2.629145,6.490476,200.747437,16.389286,3.438145
5,39.726189,30.228695,21.873341,25.137228,34.783714,25.731295,29.199146,0.0,0.0,42556.792969,32561.603516,1.183333,1.183333,0.0,2.666667,2.700337,6.455952,208.852753,16.374762,3.448766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,39.940475,29.995365,22.013222,25.119320,34.787743,26.110174,29.478672,0.0,0.0,42523.425781,33146.554688,1.227381,1.227381,0.0,2.726191,2.510261,6.545238,211.881470,16.035000,3.339752
363,38.333332,30.020958,21.890602,25.103323,34.867386,25.852745,29.385153,0.0,0.0,42526.039062,33108.386719,1.558333,1.558333,0.0,2.702381,2.553680,6.432143,211.999588,16.244644,3.389325
364,38.464287,29.948935,21.904293,25.085789,34.782036,25.894047,29.390026,0.0,0.0,42529.035156,33102.367188,1.463095,1.463095,0.0,2.714286,2.494618,6.538095,217.881134,16.136906,3.351900
